<a href="https://colab.research.google.com/github/kev1n/Fundamental-Analysis/blob/main/statanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!pip install git+https://github.com/simfin/simfin.git
import simfin as sf
from simfin.names import *
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

  Cloning https://github.com/simfin/simfin.git to /tmp/pip-req-build-z04y9e2s
  Running command git clone -q https://github.com/simfin/simfin.git /tmp/pip-req-build-z04y9e2s
  Created wheel for simfin: filename=simfin-0.8.1-cp36-none-any.whl size=60421 sha256=8e753bf86ba251c0dcee26a397ebd8e9552bbd386a4e017da027adc80b802433
  Stored in directory: /tmp/pip-ephem-wheel-cache-w1dzcnhj/wheels/dc/57/64/e9165288c478f38b172239c9f9712db2d0fcbc980e7357a3d8
Successfully built simfin


In [ ]:
# SimFin data-directory.
sf.set_data_dir('~/simfin_data/')
# SimFin load API key or use free data.
sf.load_api_key(path='~/simfin_api_key.txt', default_key='free')

# We are interested in the US stock-market.
market = 'us'

# Add this date-offset to the fundamental data such as
# Income Statements etc., because the REPORT_DATE is not
# when it was actually made available to the public,
# which can be 1, 2 or even 3 months after the Report Date.
offset = pd.DateOffset(days=60)

# Refresh the fundamental datasets (Income Statements etc.)
# every 30 days.
refresh_days = 30

# Refresh the dataset with shareprices every 10 days.
refresh_days_shareprices = 1

hub = sf.StockHub(market=market, offset=offset,
                  refresh_days=refresh_days,
                  refresh_days_shareprices=refresh_days_shareprices)

In [ ]:
df_fin_signals = hub.fin_signals(variant='daily')
df_growth_signals = hub.growth_signals(variant='daily')
df_val_signals = hub.val_signals(variant='daily')
df_returns = hub.returns(years=1, annualized=True, future= True)


Dataset "us-income-ttm" not on disk.
- Downloading ... 100.0%
- Extracting zip-file ... Done!
- Loading from disk ... Done!
Dataset "us-balance-ttm" not on disk.
- Downloading ... 100.0%
- Extracting zip-file ... Done!
- Loading from disk ... Done!
Dataset "us-cashflow-ttm" not on disk.
- Downloading ... 100.0%
- Extracting zip-file ... Done!
- Loading from disk ... Done!
Dataset "us-shareprices-daily" not on disk.
- Downloading ... 100.0%
- Extracting zip-file ... Done!
- Loading from disk ... Done!
Cache-file 'fin_signals-2a38bb7d.pickle' not on disk.
- Running function fin_signals() ... 

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


Done!
- Saving cache-file to disk ... Done!
Dataset "us-income-quarterly" not on disk.
- Downloading ... 100.0%
- Extracting zip-file ... Done!
- Loading from disk ... Done!
Dataset "us-balance-quarterly" not on disk.
- Downloading ... 100.0%
- Extracting zip-file ... Done!
- Loading from disk ... Done!
Dataset "us-cashflow-quarterly" not on disk.
- Downloading ... 100.0%
- Extracting zip-file ... Done!
- Loading from disk ... Done!
Cache-file 'growth_signals-2a38bb7d.pickle' not on disk.
- Running function growth_signals() ... Done!
- Saving cache-file to disk ... Done!
Cache-file 'val_signals-739b68a6.pickle' not on disk.
- Running function val_signals() ... Done!
- Saving cache-file to disk ... Done!
Cache-file 'rel_change-1dcdd7ff.pickle' not on disk.
- Running function rel_change() ... Done!
- Saving cache-file to disk ... Done!


In [ ]:
dfs = [df_fin_signals, df_growth_signals, df_val_signals, df_returns]
df_signals = pd.concat(dfs, axis=1)
df = df_signals.dropna(how='all')

In [ ]:
df = df_signals.dropna(how='all').reset_index(drop=True)
print((df.isnull().sum() / len(df)).sort_values(ascending=False))
# List of the columns before removing any.
columns_before = df_signals.columns

# Threshold for the number of rows that must be NaN for each column.
thresh = 0.75 * len(df_signals.dropna(how='any'))

# Remove all columns which don't have sufficient data.
df_signals = df_signals.dropna(axis='columns', thresh=thresh)
df_signals = df_signals[df_signals['Market-Cap'] > 50000000]

R&D / Revenue                        0.766018
R&D / Gross Profit                   0.765928
Return on Research Capital           0.765928
Dividend Yield                       0.701099
Net Acquisitions / Total Assets      0.694001
Inventory Turnover                   0.562529
Debt Ratio                           0.557697
Sales Growth YOY                     0.487863
Sales Growth                         0.484880
Earnings Growth YOY                  0.479990
FCF Growth YOY                       0.479747
Assets Growth YOY                    0.479747
Earnings Growth                      0.475537
FCF Growth                           0.475328
Assets Growth                        0.475328
Interest Coverage                    0.466678
Sales Growth QOQ                     0.425824
CapEx / (Depr + Amor)                0.418064
Earnings Growth QOQ                  0.415795
FCF Growth QOQ                       0.415518
Assets Growth QOQ                    0.415518
P/Sales                           

# Alpha Finding

In [ ]:
print(df_signals.columns)

import itertools
factored = (list(itertools.combinations(['(Dividends + Share Buyback) / FCF', 'Debt Ratio', 'Quick Ratio', 'Gross Profit Margin', 'Interest Coverage', 'Log Revenue',
       'Net Profit Margin', 'Return on Assets', 'Return on Equity', 'Assets Growth YOY', 'Earnings Growth YOY', 'FCF Growth YOY', 'Return on Assets',
       'Sales Growth YOY', 'FCF Yield', 'P/E', 'P/NCAV', 'Current Ratio', 'Earnings Yield',
       'Price to Book Value'], 3)))

print(len(factored))

Index(['(Dividends + Share Buyback) / FCF', 'Asset Turnover',
       'CapEx / (Depr + Amor)', 'Current Ratio', 'Debt Ratio',
       'Dividends / FCF', 'Gross Profit Margin', 'Interest Coverage',
       'Inventory Turnover', 'Log Revenue', 'Net Acquisitions / Total Assets',
       'Net Profit Margin', 'Quick Ratio', 'R&D / Gross Profit',
       'R&D / Revenue', 'Return on Assets', 'Return on Equity',
       'Return on Research Capital', 'Share Buyback / FCF', 'Assets Growth',
       'Assets Growth QOQ', 'Assets Growth YOY', 'Earnings Growth',
       'Earnings Growth QOQ', 'Earnings Growth YOY', 'FCF Growth',
       'FCF Growth QOQ', 'FCF Growth YOY', 'Sales Growth', 'Sales Growth QOQ',
       'Sales Growth YOY', 'Dividend Yield', 'Earnings Yield', 'FCF Yield',
       'Market-Cap', 'P/Cash', 'P/E', 'P/FCF', 'P/NCAV', 'P/NetNet', 'P/Sales',
       'Price to Book Value', 'Adj. Close'],
      dtype='object')
1140


In [ ]:
#Backtester
pd.options.mode.chained_assignment = None
pd.set_option("display.max_rows", None, "display.max_columns", None)
storage = {}
counter = 0
for combos in factored:
  factors = list(combos)
  yfactors = ['Adj. Close']
  allfactors = factors + ['Adj. Close']
  dates = [("2017-1-04", "2018-1-05"), ("2016-1-04", "2017-1-05"), ("2015-1-02", "2016-1-04"), ("2014-1-02", "2015-1-02"), ("2013-1-02", "2014-1-02"), ("2012-1-03", "2013-1-02"), ("2011-1-03", "2012-1-03"), ("2010-1-04", "2011-1-03"), ("2009-1-02", "2010-1-04")]
  dates2 = [("2017-6-01", "2018-6-01"), ("2016-6-01", "2017-6-01"), ("2015-6-01", "2016-6-01"), ("2014-6-02", "2015-6-02"), ("2013-6-03", "2014-6-03"), ("2012-6-01", "2013-6-03"), ("2011-6-03", "2012-6-01") , ("2010-6-04", "2011-6-03"), ("2009-6-02", "2010-6-04")]

  annualized = []
  annualized2 = []
  for i in dates:
      dfs = df_signals[allfactors].xs(i[0], level="Date")
      dfs[dfs==np.inf]=np.nan
      dfs[dfs==-np.inf]=np.nan
      datasets = dfs.dropna(axis=0)
      datasets = datasets[datasets[factors].apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]
      trainingX = datasets[factors]
      trainingY = datasets[yfactors]

      dfs = df_signals[allfactors].xs(i[1], level="Date")
      dfs[dfs==np.inf]=np.nan
      dfs[dfs==-np.inf]=np.nan
      dataset = dfs.dropna(axis=0)
      dataset = dataset[dataset[factors].apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]
      fitX = dataset[factors]
      fitY = dataset[yfactors]

      model = RandomForestRegressor(random_state=1, max_leaf_nodes=10)
      fitted = model.fit(trainingX, trainingY.values.ravel())
      testy = model.predict(fitX)

      fitY['Prediction'] = testy

      predictions = fitY.sort_values(by='Prediction', ascending=False)

      #annualized.append(predictions.iloc[-25:-1]['Adj. Close'].mean())
      annualized.append(predictions.iloc[0:20]['Adj. Close'].mean())
  advantage = np.prod([1 + num for num in annualized])
  advantage = advantage/ np.std(annualized)

  for i in dates2:
      dfs2 = df_signals[allfactors].xs(i[0], level="Date")
      dfs2[dfs2==np.inf]=np.nan
      dfs2[dfs2==-np.inf]=np.nan
      datasets2 = dfs2.dropna(axis=0)
      datasets2 = datasets2[datasets2[factors].apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]
      trainingX2 = datasets2[factors]
      trainingY2 = datasets2[yfactors]

      dfs2 = df_signals[allfactors].xs(i[1], level="Date")
      dfs2[dfs2==np.inf]=np.nan
      dfs2[dfs2==-np.inf]=np.nan
      dataset2 = dfs2.dropna(axis=0)
      dataset2 = dataset2[dataset2[factors].apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]
      fitX2 = dataset2[factors]
      fitY2 = dataset2[yfactors]

      model2 = RandomForestRegressor(random_state=1, max_leaf_nodes=10)
      fitted2 = model2.fit(trainingX2, trainingY2.values.ravel())
      testy2 = model2.predict(fitX2)

      fitY2['Prediction2'] = testy2

      predictions2 = fitY2.sort_values(by='Prediction2', ascending=False)

      #annualized.append(predictions.iloc[-25:-1]['Adj. Close'].mean())
      annualized2.append(predictions2.iloc[0:20]['Adj. Close'].mean())
  advantage2 = np.prod([1 + num for num in annualized2])
  advantage2 = advantage2/ np.std(annualized)

  counter += 1
  # if advantage > advantage2:
  #   fourth = ((advantage - advantage2) * 0.25) + advantage2
  # else:
  #   fourth = ((advantage2 - advantage) * 0.25) + advantage
  fourth = np.mean([advantage, advantage2])
  storage[str(factors)] = fourth
  dicts = {k: v for k, v in sorted(storage.items(), key=lambda item: item[1], reverse=True)}
  value_at_index = list(dicts.values())[0], list(dicts.keys())[0]
  print(value_at_index, counter)



(23.371370091816043, "['(Dividends + Share Buyback) / FCF', 'Debt Ratio', 'Quick Ratio']") 1
(23.371370091816043, "['(Dividends + Share Buyback) / FCF', 'Debt Ratio', 'Quick Ratio']") 2
(23.371370091816043, "['(Dividends + Share Buyback) / FCF', 'Debt Ratio', 'Quick Ratio']") 3
(23.371370091816043, "['(Dividends + Share Buyback) / FCF', 'Debt Ratio', 'Quick Ratio']") 4
(23.371370091816043, "['(Dividends + Share Buyback) / FCF', 'Debt Ratio', 'Quick Ratio']") 5
(23.93496423012971, "['(Dividends + Share Buyback) / FCF', 'Debt Ratio', 'Return on Assets']") 6
(40.133035682136665, "['(Dividends + Share Buyback) / FCF', 'Debt Ratio', 'Return on Equity']") 7
(40.133035682136665, "['(Dividends + Share Buyback) / FCF', 'Debt Ratio', 'Return on Equity']") 8
(40.133035682136665, "['(Dividends + Share Buyback) / FCF', 'Debt Ratio', 'Return on Equity']") 9
(40.133035682136665, "['(Dividends + Share Buyback) / FCF', 'Debt Ratio', 'Return on Equity']") 10
(40.133035682136665, "['(Dividends + Share Bu

In [ ]:
print(dicts)

{"['(Dividends + Share Buyback) / FCF', 'Debt Ratio', 'Return on Equity']": 40.133035682136665, "['Quick Ratio', 'P/NCAV', 'Current Ratio']": 35.271243898486446, "['Debt Ratio', 'Gross Profit Margin', 'Return on Equity']": 33.053976407768324, "['(Dividends + Share Buyback) / FCF', 'Quick Ratio', 'Price to Book Value']": 32.48276557991788, "['Debt Ratio', 'FCF Growth YOY', 'P/E']": 30.188773786892646, "['Debt Ratio', 'Gross Profit Margin', 'Interest Coverage']": 29.51731245186579, "['Interest Coverage', 'Return on Equity', 'Sales Growth YOY']": 29.16186101848247, "['Gross Profit Margin', 'P/E', 'P/NCAV']": 28.42552520873994, "['(Dividends + Share Buyback) / FCF', 'Debt Ratio', 'Current Ratio']": 28.227036374133306, "['(Dividends + Share Buyback) / FCF', 'FCF Yield', 'Price to Book Value']": 28.07207723371682, "['Gross Profit Margin', 'FCF Yield', 'P/NCAV']": 28.034189349909518, "['FCF Yield', 'P/NCAV', 'Current Ratio']": 27.98820630006351, "['Debt Ratio', 'Interest Coverage', 'P/NCAV']"

In [ ]:
#Backtester
pd.options.mode.chained_assignment = None
pd.set_option("display.max_rows", None, "display.max_columns", None)
storage = {}
counter = 0
for combos in factored:
  factors = list(combos)
  yfactors = ['Adj. Close']
  allfactors = factors + ['Adj. Close']
  dates = [("2017-1-04", "2018-1-05"), ("2016-1-04", "2017-1-05"), ("2015-1-02", "2016-1-04"), ("2014-1-02", "2015-1-02"), ("2013-1-02", "2014-1-02"), ("2012-1-03", "2013-1-02"), ("2011-1-03", "2012-1-03"), ("2010-1-04", "2011-1-03"), ("2009-1-02", "2010-1-04")]
  dates2 = [("2017-6-01", "2018-6-01"), ("2016-6-01", "2017-6-01"), ("2015-6-01", "2016-6-01"), ("2014-6-02", "2015-6-02"), ("2013-6-03", "2014-6-03"), ("2012-6-01", "2013-6-03"), ("2011-6-03", "2012-6-01") , ("2010-6-04", "2011-6-03"), ("2009-6-02", "2010-6-04")]

  annualized = []
  annualized2 = []
  for i in dates:
      dfs = df_signals[allfactors].xs(i[0], level="Date")
      dfs[dfs==np.inf]=np.nan
      dfs[dfs==-np.inf]=np.nan
      datasets = dfs.dropna(axis=0)
      datasets = datasets[datasets[factors].apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]
      trainingX = datasets[factors]
      trainingY = datasets[yfactors]

      dfs = df_signals[allfactors].xs(i[1], level="Date")
      dfs[dfs==np.inf]=np.nan
      dfs[dfs==-np.inf]=np.nan
      dataset = dfs.dropna(axis=0)
      dataset = dataset[dataset[factors].apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]
      fitX = dataset[factors]
      fitY = dataset[yfactors]

      model = RandomForestRegressor(random_state=1, max_leaf_nodes=10)
      fitted = model.fit(trainingX, trainingY.values.ravel())
      testy = model.predict(fitX)

      fitY['Prediction'] = testy

      predictions = fitY.sort_values(by='Prediction', ascending=False)

      #annualized.append(predictions.iloc[-25:-1]['Adj. Close'].mean())
      annualized.append(predictions.iloc[0:13]['Adj. Close'].mean())
  advantage = np.prod([1 + num for num in annualized])

  counter += 1


  storage[str(factors)] = advantage
  dicts = {k: v for k, v in sorted(storage.items(), key=lambda item: item[1], reverse=True)}
  value_at_index = list(dicts.values())[0], list(dicts.keys())[0]
  print(value_at_index, counter)

In [ ]:
print(dicts)

# Alpha Refining

In [ ]:
#Alpha refiner
pd.options.mode.chained_assignment = None

factors = ['Debt Ratio', 'P/NCAV', 'Current Ratio']
yfactors = ['Adj. Close']
allfactors = factors + ['Adj. Close']
dates = [("2017-1-04", "2018-1-05"), ("2016-1-04", "2017-1-05"), ("2015-1-02", "2016-1-04"), ("2014-1-02", "2015-1-02"), ("2013-1-02", "2014-1-02"), ("2012-1-03", "2013-1-02"), ("2011-1-03", "2012-1-03") , ("2010-1-04", "2011-1-03"), ("2009-1-02", "2010-1-04"), ("2008-1-02", "2009-1-02")]
dates2 = [("2017-6-01", "2018-6-01"), ("2016-6-01", "2017-6-01"), ("2015-6-01", "2016-6-01"), ("2014-6-02", "2015-6-02"), ("2013-6-03", "2014-6-03"), ("2012-6-01", "2013-6-03"), ("2011-6-03", "2012-6-01") , ("2010-6-04", "2011-6-03"), ("2009-6-02", "2010-6-04"), ("2008-6-02", "2009-6-02")]

yes = []
no = []
for z in range(2, 25):
  meaned = []
  for i in dates:
    dfs = df_signals[allfactors].xs(i[0], level="Date")
    dfs[dfs==np.inf]=np.nan
    dfs[dfs==-np.inf]=np.nan
    datasets = dfs.dropna(axis=0)
    datasets = datasets[datasets[factors].apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]
    trainingX = datasets[factors]
    trainingY = datasets[yfactors]

    dfs = df_signals[allfactors].xs(i[1], level="Date")
    dfs[dfs==np.inf]=np.nan
    dfs[dfs==-np.inf]=np.nan
    dataset = dfs.dropna(axis=0)
    dataset = dataset[dataset[factors].apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]
    fitX = dataset[factors]
    fitY = dataset[yfactors]

    model = RandomForestRegressor(random_state=1, max_leaf_nodes=z)
    fitted = model.fit(trainingX, trainingY.values.ravel())
    testy = model.predict(fitX)

    fitY['Prediction'] = testy

    pd.set_option("display.max_rows", None, "display.max_columns", None)
    predictions = fitY.sort_values(by='Prediction', ascending=False)
    #print(predictions.describe())

    meaned.append(predictions.iloc[0:13]['Adj. Close'].mean())
    #annualized.append(predictions.iloc[-25:-1]['Adj. Close'].mean())
    #print(predictions.iloc[0:25])
    #print(i[1])
    #print(predictions['Adj. Close'].iloc[0:z].mean(), i[1])
    #print('\n')
  prod1 = np.product([x+1 for x in meaned])
  yes.append(prod1)
print(meaned)

for z in range(2, 25):
    
  meaned = []
  for i in dates2:
    dfs = df_signals[allfactors].xs(i[0], level="Date")
    dfs[dfs==np.inf]=np.nan
    dfs[dfs==-np.inf]=np.nan
    datasets = dfs.dropna(axis=0)
    datasets = datasets[datasets[factors].apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]
    trainingX = datasets[factors]
    trainingY = datasets[yfactors]

    dfs = df_signals[allfactors].xs(i[1], level="Date")
    dfs[dfs==np.inf]=np.nan
    dfs[dfs==-np.inf]=np.nan
    dataset = dfs.dropna(axis=0)
    dataset = dataset[dataset[factors].apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]
    fitX = dataset[factors]
    fitY = dataset[yfactors]

    model = RandomForestRegressor(random_state=1, max_leaf_nodes=z)
    fitted = model.fit(trainingX, trainingY.values.ravel())
    testy = model.predict(fitX)

    fitY['Prediction'] = testy

    pd.set_option("display.max_rows", None, "display.max_columns", None)
    predictions = fitY.sort_values(by='Prediction', ascending=False)
    #print(predictions.describe())

    meaned.append(predictions.iloc[0:13]['Adj. Close'].mean())
    #annualized.append(predictions.iloc[-25:-1]['Adj. Close'].mean())
    #print(predictions.iloc[0:25])
    #print(i[1])
    #print(predictions['Adj. Close'].iloc[0:z].mean(), i[1])
    #print('\n')
  prod2 = np.product([x+1 for x in meaned])
  no.append(prod2)
print(meaned)
print(no)
listed = np.array(yes) + np.array(no)
for i in range(23):
  print(listed[i])
  print(i+2)

# Date Testers


In [ ]:
#Single Date Tester
pd.options.mode.chained_assignment = None
factors = ['(Dividends + Share Buyback) / FCF', 'Debt Ratio', 'Return on Equity']
yfactors = ['Adj. Close']
allfactors = factors + ['Adj. Close']
test = [("2018-6-01", "2019-6-03")]
dates = [("2017-1-04", "2018-1-05"), ("2016-1-04", "2017-1-05"), ("2015-1-02", "2016-1-04"), ("2014-1-02", "2015-1-02"), ("2013-1-02", "2014-1-02"), ("2012-1-03", "2013-1-02"), ("2011-1-03", "2012-1-03") , ("2010-1-04", "2011-1-03"), ("2009-1-02", "2010-1-04"), ("2008-1-02", "2009-1-02")]
dates2 = [("2017-6-01", "2018-6-01"), ("2016-6-01", "2017-6-01"), ("2015-6-01", "2016-6-01"), ("2014-6-02", "2015-6-02"), ("2013-6-03", "2014-6-03"), ("2012-6-01", "2013-6-03"), ("2011-6-03", "2012-6-01") , ("2010-6-04", "2011-6-03"), ("2009-6-02", "2010-6-04"), ("2008-6-02", "2009-6-02")]
meaned = []
for i in test:
      dfs = df_signals[allfactors].xs(i[0], level="Date")
      dfs[dfs==np.inf]=np.nan
      dfs[dfs==-np.inf]=np.nan
      datasets = dfs.dropna(axis=0)
      datasets = datasets[datasets[factors].apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]
      trainingX = datasets[factors]
      trainingY = datasets[yfactors]

      dfs = df_signals[allfactors].xs(i[1], level="Date")

      dfs[dfs==np.inf]=np.nan
      dfs[dfs==-np.inf]=np.nan
      dataset = dfs.dropna(axis=0)
      dataset = dataset[dataset[factors].apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]

      fitX = dataset[factors]
      fitY = dataset[yfactors]
    

      model = RandomForestRegressor(random_state=1, max_leaf_nodes=18)
    
      fitted = model.fit(trainingX, trainingY.values.ravel())
      testy = model.predict(fitX)

      fitY['Prediction'] = testy
  
      pd.set_option("display.max_rows", None, "display.max_columns", None)
      predictions = fitY.sort_values(by='Prediction', ascending=False)
      print(predictions[0:8], i[1], predictions.iloc[0:8]['Adj. Close'].mean())
      #print(predictions.describe())

      meaned.append(predictions.iloc[0:8]['Adj. Close'].mean())

      #annualized.append(predictions.iloc[-25:-1]['Adj. Close'].mean())
      #print(predictions.iloc[0:25])
      #print(i[1])
      #print(predictions['Adj. Close'].iloc[0:25].mean())
      #print('\n')

prod1 = np.product([x+1 for x in meaned])
print(prod1)

ValueError: ignored

In [ ]:
#Double Date Tester
def tester(max_leaf_nodes, numstocks, n_estimators, max_depth, factors):
  pd.options.mode.chained_assignment = None
  factors = factors
  yfactors = ['Adj. Close']
  allfactors = factors + ['Adj. Close']
  test = [("2018-6-01", "2019-6-03")]
  dates = [("2017-1-04", "2018-1-05"), ("2016-1-04", "2017-1-05"), ("2015-1-02", "2016-1-04"), ("2014-1-02", "2015-1-02"), ("2013-1-02", "2014-1-02"), ("2012-1-03", "2013-1-02"), ("2011-1-03", "2012-1-03") , ("2010-1-04", "2011-1-03"), ("2009-1-02", "2010-1-04")]
  dates2 = [("2017-6-01", "2018-6-01"), ("2016-6-01", "2017-6-01"), ("2015-6-01", "2016-6-01"), ("2014-6-02", "2015-6-02"), ("2013-6-03", "2014-6-03"), ("2012-6-01", "2013-6-03"), ("2011-6-03", "2012-6-01") , ("2010-6-04", "2011-6-03"), ("2009-6-02", "2010-6-04")]
  meaned = []
  special = [("2017-6-01", "2018-6-01")]
  for i in dates:
        dfs = df_signals[allfactors].xs(i[0], level="Date")
        dfs[dfs==np.inf]=np.nan
        dfs[dfs==-np.inf]=np.nan
        datasets = dfs.dropna(axis=0)
        datasets = datasets[datasets[factors].apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]
        trainingX = datasets[factors]
        trainingY = datasets[yfactors]

        dfs = df_signals[allfactors].xs(i[1], level="Date")
        
        dfs[dfs==np.inf]=np.nan
        dfs[dfs==-np.inf]=np.nan
        dataset = dfs.dropna(axis=0)
        dataset = dataset[dataset[factors].apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]

        fitX = dataset[factors]
        fitY = dataset[yfactors]
      

        model = RandomForestRegressor(random_state=1, max_leaf_nodes=max_leaf_nodes, n_estimators=n_estimators, max_depth=max_depth)
      
        fitted = model.fit(trainingX, trainingY.values.ravel())
        testy = model.predict(fitX)

        fitY['Prediction'] = testy
    
        pd.set_option("display.max_rows", None, "display.max_columns", None)
        predictions = fitY.sort_values(by='Prediction', ascending=False)
        print(predictions[0:numstocks], i[1], predictions.iloc[0:numstocks]['Adj. Close'].mean())
        meaned.append(predictions.iloc[0:numstocks]['Adj. Close'].mean())


  prod1 = np.product([x+1 for x in meaned])
  print(prod1)
  print('--------------------------')
  meaned = []
  for i in dates2:
        dfs = df_signals[allfactors].xs(i[0], level="Date")
        dfs[dfs==np.inf]=np.nan
        dfs[dfs==-np.inf]=np.nan
        datasets = dfs.dropna(axis=0)
        datasets = datasets[datasets[factors].apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]
        trainingX = datasets[factors]
        trainingY = datasets[yfactors]

        dfs = df_signals[allfactors].xs(i[1], level="Date")

        dfs[dfs==np.inf]=np.nan
        dfs[dfs==-np.inf]=np.nan
        dataset = dfs.dropna(axis=0)
        dataset = dataset[dataset[factors].apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]

        fitX = dataset[factors]
        fitY = dataset[yfactors]
      

        model = RandomForestRegressor(random_state=1, max_leaf_nodes=max_leaf_nodes, n_estimators=n_estimators, max_depth=max_depth)
      
        fitted = model.fit(trainingX, trainingY.values.ravel())
        testy = model.predict(fitX)

        fitY['Prediction2'] = testy
    
        pd.set_option("display.max_rows", None, "display.max_columns", None)
        predictions = fitY.sort_values(by='Prediction2', ascending=False)
        print(predictions[0:numstocks], i[1], predictions.iloc[0:numstocks]['Adj. Close'].mean())
        meaned.append(predictions.iloc[0:numstocks]['Adj. Close'].mean())

  prod1 = np.product([x+1 for x in meaned])
  print(prod1)

tester(15, 10, 250, 50, ['Debt Ratio', 'P/E', 'Current Ratio'])
#tester(50, 20, 300, 50, ['(Dividends + Share Buyback) / FCF', 'Debt Ratio', 'Return on Equity'])


        Adj. Close  Prediction
Ticker                        
DVAX     -0.450380    0.772452
TROX     -0.548470    0.732159
PACB      1.657242    0.682853
HALO     -0.211984    0.616983
KURA      0.102200    0.606170
ARWR      1.973162    0.604454
INCY     -0.249592    0.580945
CDNA      2.641613    0.567368
CRR      -0.615362    0.523835
PODD      0.078282    0.497156 2018-1-05 0.43767125141085206
        Adj. Close  Prediction
Ticker                        
TDG       0.184041    3.676068
REN      -0.192880    3.249552
EAT      -0.145151    2.914997
TRHC      1.231277    2.858434
WFT      -0.330098    1.706734
KND       0.212565    1.660122
NRG       1.174761    1.603535
SWN      -0.457898    1.549851
VSM       0.400830    1.465737
TLRD     -0.007386    1.265096 2017-1-05 0.2070059188888346
        Adj. Close  Prediction
Ticker                        
EXEL      1.679595    0.796909
NERV      0.996489    0.636720
CBB       0.379535    0.570845
CQP       0.210361    0.565993
REN       8

# Signals

In [ ]:
def signals(max_leaf_nodes, numstocks, n_estimators, max_depth, factors):
  pd.options.mode.chained_assignment = None
  factors = factors
  yfactors = ['Adj. Close']
  allfactors = factors + ['Adj. Close']
  
  dates = [("2017-1-04", "2018-1-05"), ("2016-1-04", "2017-1-05"), ("2015-1-02", "2016-1-04"), ("2014-1-02", "2015-1-02"), ("2013-1-02", "2014-1-02"), ("2012-1-03", "2013-1-02"), ("2011-1-03", "2012-1-03") , ("2010-1-04", "2011-1-03"), ("2009-1-02", "2010-1-04"), ("2008-1-02", "2009-1-02")]
  dates2 = [("2017-6-01", "2018-6-01"), ("2016-6-01", "2017-6-01"), ("2015-6-01", "2016-6-01"), ("2014-6-02", "2015-6-02"), ("2013-6-03", "2014-6-03"), ("2012-6-01", "2013-6-03"), ("2011-6-03", "2012-6-01") , ("2010-6-04", "2011-6-03"), ("2009-6-02", "2010-6-04"), ("2008-6-02", "2009-6-02")]
  special = [("2017-6-01", "2018-6-01")]
  almostnow = [("2018-9-20", "2019-9-20")]
  test = [("2018-3-01", "2019-3-06")]
  test2 = [("2018-1-05", "2019-1-03")]
  test3 = [("2018-3-20", "2019-3-20")]
  test4= [("2018-3-20", "2019-10-18")]
  test5=[("2018-8-30", "2019-8-28")]
  test6=[("2018-4-05", "2019-5-09")]

  test8=[("2018-11-12", "2019-11-15")]
  meaned = []
  for i in test8:
        dfs = df_signals[allfactors].xs(i[0], level="Date")
        dfs[dfs==np.inf]=np.nan
        dfs[dfs==-np.inf]=np.nan
        datasets = dfs.dropna(axis=0)
        datasets = datasets[datasets[factors].apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]
        trainingX = datasets[factors]
        trainingY = datasets[yfactors]

        dfs = df_signals[allfactors].xs(i[1], level="Date")
        
        dfs[dfs==np.inf]=np.nan
        dfs[dfs==-np.inf]=np.nan
        dataset = dfs[factors].dropna(axis=0)
        dataset = dataset[dataset[factors].apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]

        fitX = dataset[factors]

        model = RandomForestRegressor(random_state=1, max_leaf_nodes=max_leaf_nodes, n_estimators=n_estimators, max_depth=max_depth)
      
        fitted = model.fit(trainingX, trainingY.values.ravel())

        testy = model.predict(fitX)
        fitX['Prediction'] = testy
  predictions = fitX.sort_values(by='Prediction', ascending=False)
  print(predictions[0:numstocks])

signals(15, 10, 250, 50, ['Debt Ratio', 'P/E', 'Current Ratio'])

        Debt Ratio         P/E  Current Ratio  Prediction
Ticker                                                   
PAYS      0.000000  146.865871       1.229267    0.843948
AXGN      0.026902  -24.342364       8.703998    0.804941
DRQ       0.000041  -18.130988       9.967542    0.662027
HSII      0.000000   10.677778       1.489244    0.592568
ROCK      0.000000   29.344976       1.918934    0.318082
CWH       0.776108  -56.156764       1.353350    0.299982
DXCM      0.524164 -109.226210       6.052047    0.285970
SRPT      0.352352  -11.986284       8.806384    0.284993
BYND      0.077837  -36.454828       8.954682    0.252926
UTMD      0.000000   22.196733      12.097618    0.220177
